In [ ]:
# install semantic-link-labs
%pip install semantic-link-labs

In [ ]:
Param_Lakehouse= "" #name or ID of the lakehouse where you want to save the #.pbip file
Param_LakehouseWorkspace ="" #name or ID of the workspace in which the lakehouse exists

In [ ]:
import sempy.fabric as fabric
import pandas as pd
import sempy_labs.report as rep

# Get List of all workspaces
df_workspaces = fabric.list_workspaces().query('Type != "AdminInsights"')



# Iterate over workspaces and fetch reports
all_reports = []

for _, row in df_workspaces.iterrows():
    workspace_id = row['Id']
    workspace_name = row['Name']
    
    try:
        df_reports = fabric.list_reports(workspace_id)
        if not df_reports.empty:
            df_reports['ReportWorkspaceName'] = workspace_name
            df_reports['ReportWorkspaceId'] = workspace_id
            all_reports.append(df_reports)
    except Exception as e:
        print(f"Error fetching reports for workspace '{workspace_name}': {e}")

# Combine all reports into one DataFrame
if all_reports:
    df_all_reports = pd.concat(all_reports, ignore_index=True)

# If set to True, saves the report and underlying semantic model. If # set to False, saves just the report.
thick_report = True 

# If set to True, saves a .pbip live-connected to the workspace in 
# the Power BI / Fabric service. If set to False, saves a .pbip with  # a local model, independent from the Power BI / Fabric service.
live_connect = True 


for _, row in df_all_reports.iterrows():
    rep.save_report_as_pbip(report=row["Id"], workspace=row["ReportWorkspaceId"], thick_report=thick_report, live_connect=live_connect, lakehouse=Param_Lakehouse, lakehouse_workspace=Param_LakehouseWorkspace)

